In [57]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
from sklearn.metrics import mean_squared_error
#matplotlib.rcParams['text.color'] = 'G'
#df = pd.read_excel("BU_MI_TR.xlsx")
df = pd.read_excel("2_Сглаженный исхдник дял прогноза_сглаживание рядов.xlsx")
y = df.set_index(["create_date"])
y=y[y['Region'] != 'Горно-Алтайск']
#y=y[y['Region'] == 'Архангельск']
y=y[['Region', 'fin']]
y.columns = ['Region', 'nps']
#for tr
#y=y[['nps']]
#y['Region'] = 'TR'
y


,Region,nps
create_date,,
2019-01-01,Архангельск,0.253886
2019-02-01,Архангельск,0.242188
2019-03-01,Архангельск,0.224670
2019-04-01,Архангельск,0.266667
2019-05-01,Архангельск,0.240642
...,...,...
2022-10-01,Ярославль,0.326633
2022-11-01,Ярославль,0.464646
2022-12-01,Ярославль,0.445977


In [58]:
#подбираем параметры для модели
p = d = q = range(0, 2)
PDQ = list(itertools.product(p, d, q))
SEASONAL_PDQ = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter for SARIMA...')
print('SARIMAX: {} x {}'.format(PDQ[1], SEASONAL_PDQ[1]))
print('SARIMAX: {} x {}'.format(PDQ[1], SEASONAL_PDQ[2]))
print('SARIMAX: {} x {}'.format(PDQ[2], SEASONAL_PDQ[3]))
print('SARIMAX: {} x {}'.format(PDQ[2], SEASONAL_PDQ[4]))


Examples of parameter for SARIMA...
SARIMAX: (0, 0, 1) x (0, 0, 1, 12)
SARIMAX: (0, 0, 1) x (0, 1, 0, 12)
SARIMAX: (0, 1, 0) x (0, 1, 1, 12)
SARIMAX: (0, 1, 0) x (1, 0, 0, 12)


In [59]:
def reg_forcust(y, region):
    #запускаем перебеор
    ''' for param in PDQ:
        for param_seasonal in SEASONAL_PDQ:
            try:
                mod = sm.tsa.statespace.SARIMAX(y,order=param,seasonal_order=param_seasonal,enforce_stationarity=False,enforce_invertibility=False)
                results = mod.fit()
                print('ARIMA{}x{}12 - AIC:{}'.format(param,param_seasonal,results.aic))
            except: 
                continue
    # распечатываем саммари'''
    mod = sm.tsa.statespace.SARIMAX(y,
                                    order=(1, 1, 1),
                                    seasonal_order=(1, 1, 1,12 ),
                                    #enforce_stationarity=False,
                                    enforce_invertibility=False)
    results = mod.fit()
    #print(results.summary().tables[1])

    pred = results.get_prediction(start=pd.to_datetime('2022-03-01'), dynamic=False)
    #pd.DataFrame(pred.predicted_mean) 
    y_forecasted = pred.predicted_mean
    y_truth = y['2022-03-01':]
    #print(mean_squared_error(y_truth.nps , y_forecasted))
    #прогноз
    pred_uc = results.get_forecast(steps=12)
    forecast = pred_uc.predicted_mean.head(12)
    forecast = pd.DataFrame(forecast)
    forecast['region'] = region
    return forecast

In [ ]:
total_df = pd.DataFrame()
for region in y['Region'].unique().tolist():
    #print(region)
    tmp_df = y[y['Region'] == region]
    tmp_df.drop('Region', axis= 1 , inplace= True )
    tmp_df.head()
    region_forecast_df = reg_forcust(tmp_df, region)
    total_df = pd.concat([total_df, region_forecast_df])
    
print(total_df)

In [11]:
total_df.to_excel('forecast_region.xlsx')

In [63]:
#for TR
total_df.columns = ['nps', 'Region']
total = pd.concat([y, total_df])
total

,Region,nps
2019-01-01,Архангельск,0.253886
2019-02-01,Архангельск,0.242188
2019-03-01,Архангельск,0.224670
2019-04-01,Архангельск,0.266667
2019-05-01,Архангельск,0.240642
...,...,...
2023-10-01,Ярославль,0.365877
2023-11-01,Ярославль,0.450711
2023-12-01,Ярославль,0.437526
2024-01-01,Ярославль,0.338919


In [64]:
total.to_excel('2_!forecast_all_reg.xlsx')